# SCYLLER - EXPLORATORY & SIMULATION TOOL OF CONTAMINATION DISPERSION

### Welcome, wanderer of knowledge of the oceans. This is SCYLLER, a tool developed under the scope of the ULisses project 2025, to conduct exploratory and numerical simulation testing on COPERNICUS DATA (currents & surface waves) to understand the behaviour of spilled plastic pellets.

### This tool was developed by Raúl Galdeano Pazos - Scylla Team Member & Data Wizard

### If you have any questions, contact me in this email raulgaldeanopazos@gmail.com

In [18]:
# === 🌊 Ross Sea Particle Drift Simulator ===
# Author: Raúl Galdeano Pazos · July 2025
# Exploratory map to know if there are nearby salvage parties.
# This is the libraries you need to use to run Scyller, our tool: 
from time import time 
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib import gridspec
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from scipy.interpolate import RegularGridInterpolator
import folium
from geopy.distance import geodesic
import seaborn as sns
import pandas as pd
from scipy.stats import gaussian_kde


## SCYLLER - GEOGRAPHICAL TOOL TO KNOW WHAT HARBOURS ARE NEARBY

In [ ]:

# Exploratory map to know if there are nearby salvage parties.

# To set coordinates:

shipwreck_coords = [-60, 170]  # If you have a shipwreck, of course :)

# Ports or stations we know nearby the Ross Sea:

ports = {
    "Otago Harbour": [-45.82635, 170.62165], # New Zealand 
    "Bathurst Harbour": [-43.29757, 146.17765],
    "McMurdo Station": [-77.84225, 166.67343],
}

# Create the map:

map_ross = folium.Map(location=shipwreck_coords, zoom_start = 3, tiles='OpenStreetMap')

# Add red shipwreck marker:

folium.Marker(
    location=shipwreck_coords,
    popup="Shipwreck Location",
    icon=folium.Icon(color='red', icon='ship', prefix='fa')
).add_to(map_ross)

# Loop through ports:

for name, coords in ports.items():
    # Add port marker: 
    folium.Marker(
        location=coords,
        popup=f"{name}",
        icon=folium.Icon(color='purple', icon='anchor', prefix='fa')
    ).add_to(map_ross)

    # Draw line to shipwreck:

    folium.PolyLine(locations=[coords, shipwreck_coords], color='green', weight=3.5).add_to(map_ross)

    # Calculate distance:

    dist_km = geodesic(coords, shipwreck_coords).kilometers

    # Add distance as a popup on the midpoint of the line:

    midpoint = [
        (coords[0] + shipwreck_coords[0]) / 2,
        (coords[1] + shipwreck_coords[1]) / 2 + 5
    ]
    folium.Marker(
        location=midpoint,
        icon=folium.DivIcon(html=f"<div style='font-size: 7pt; color: black;'>{dist_km:.1f} km</div>")
    ).add_to(map_ross)

# Display: 

map_ross


## SCYLLER - EXPLORATORY MAP TO UNDERSTAND THE DISPERSION

In [21]:
# === 1. Load NetCDF dataset with ocean currents ===
ds = xr.open_dataset("cmems_mod_glo_phy-cur_anfc_0.083deg_PT6H-i_1752432032568.nc")

# === 2. Define simulation parameters ===
lat_min, lat_max = -70, -60
lon_min, lon_max = -170, -140
n_bags = 150
dt = 21600
n_steps = ds.dims['time']
lats = ds["latitude"].values
lons = ds["longitude"].values

# === 3. Initialize particle positions randomly within the Ross Sea box ===
np.random.seed(42)
particles_lat = np.random.uniform(lat_min, lat_max, size=n_bags)
particles_lon = np.random.uniform(lon_min, lon_max, size=n_bags)

traj_lats = np.zeros((n_bags, n_steps))
traj_lons = np.zeros((n_bags, n_steps))
traj_lats[:, 0] = particles_lat
traj_lons[:, 0] = particles_lon

# === 4. Lagrangian simulation loop ===
for t in range(1, n_steps):
    uo = ds["uo"].isel(time=t-1, depth=0).values
    vo = ds["vo"].isel(time=t-1, depth=0).values
    interp_u = RegularGridInterpolator((lats, lons), uo, bounds_error=False, fill_value=np.nan)
    interp_v = RegularGridInterpolator((lats, lons), vo, bounds_error=False, fill_value=np.nan)
    coords = np.stack((particles_lat, particles_lon), axis=-1)
    u_vals = interp_u(coords)
    v_vals = interp_v(coords)
    mask = ~np.isnan(u_vals) & ~np.isnan(v_vals)

    delta_lat = np.zeros_like(particles_lat)
    delta_lon = np.zeros_like(particles_lon)
    delta_lat[mask] = (v_vals[mask] * dt) / 111320
    delta_lon[mask] = (u_vals[mask] * dt) / (40075000 * np.cos(np.radians(particles_lat[mask])) / 360)

    particles_lat[mask] += delta_lat[mask]
    particles_lon[mask] += delta_lon[mask]

    traj_lats[:, t] = particles_lat
    traj_lons[:, t] = particles_lon

# === 5. Setup KDE Grid ===
grid_lon = np.linspace(-180, -130, 200)
grid_lat = np.linspace(-80, -55, 200)
lon_grid, lat_grid = np.meshgrid(grid_lon, grid_lat)
grid_coords = np.vstack([lon_grid.ravel(), lat_grid.ravel()])

# === 6. Create animated KDE heatmap on polar map ===
fig = plt.figure(figsize=(10, 10))
ax = plt.axes(projection=ccrs.SouthPolarStereo())
ax.set_extent([-180, 180, -85, -55], crs=ccrs.PlateCarree())

ax.add_feature(cfeature.LAND, facecolor='cornsilk', zorder=0)
ax.add_feature(cfeature.OCEAN, facecolor='lightblue', zorder=0)
ax.coastlines(resolution='110m', linewidth=0.8)
ax.gridlines(draw_labels=False, linestyle=":", linewidth=0.5, color="gray")

kde_img = None
title = ax.set_title("", fontsize=16, fontfamily="arial", fontweight="bold")

def animate(t):
    global kde_img
    ax.clear()
    ax.set_extent([-180, 180, -85, -55], crs=ccrs.PlateCarree())
    ax.add_feature(cfeature.LAND, facecolor='cornsilk', zorder=0)
    ax.add_feature(cfeature.OCEAN, facecolor='lightblue', zorder=0)
    ax.coastlines(resolution='110m', linewidth=0.8)
    ax.gridlines(draw_labels=False, linestyle=":", linewidth=0.5, color="gray")

    # KDE calculation
    kde = gaussian_kde(np.vstack([traj_lons[:, t], traj_lats[:, t]]))
    kde_vals = kde(grid_coords).reshape(lon_grid.shape)

    kde_img = ax.pcolormesh(
        lon_grid,
        lat_grid,
        kde_vals,
        cmap="inferno",
        shading="auto",
        alpha=0.85,
        transform=ccrs.PlateCarree(),
        zorder=4
    )

    ax.set_title(f"🔥 Bag Density – Step {t}", fontsize=16, fontweight='bold', color='darkred')

ani = animation.FuncAnimation(fig, animate, frames=n_steps, interval=500, blit=False)
ani.save("Ross_Sea_Bags_Density.gif", writer=animation.PillowWriter(fps=2))
plt.close(fig)

print("✅ Animation saved as 'Ross_Sea_Bags_Density.gif'")


C:\Users\raul.galdeano.ext\AppData\Local\Temp\ipykernel_17072\760542395.py:9: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  n_steps = ds.dims['time']
c:\Users\raul.galdeano.ext\AppData\Local\anaconda3\envs\MOHID_Lagrangian_environment\Lib\site-packages\cartopy\mpl\geoaxes.py:527: UserWarning: Glyph 128293 (\N{FIRE}) missing from font(s) DejaVu Sans.
  super()._update_title_position(renderer)
c:\Users\raul.galdeano.ext\AppData\Local\anaconda3\envs\MOHID_Lagrangian_environment\Lib\site-packages\cartopy\mpl\geoaxes.py:524: UserWarning: Glyph 128293 (\N{FIRE}) missing from font(s) DejaVu Sans.
  return super().draw(renderer=renderer, **kwargs)
c:\Users\raul.galdeano.ext\AppData\Local\anaconda3\envs\MOHID_Lagrangian_environment\Lib\site-packages\cartopy\mpl\geoaxes.py:524: UserWarning: Glyph 

✅ Animation saved as 'Ross_Sea_Bags_Density.gif'


In [22]:
# === 🌊 Ross Sea Particle Drift Simulator ===
# Author: Raúl Galdeano Pazos · July 2025
# Simulates drift of particles in the Ross Sea using CMEMS data
# and generates a heatmap animation using KDE + Cartopy

# === 1. Load NetCDF dataset with ocean currents ===
ds = xr.open_dataset("cmems_mod_glo_phy-cur_anfc_0.083deg_PT6H-i_1752432032568.nc")

# === 2. Define simulation parameters ===
lat_min, lat_max = -70, -60
lon_min, lon_max = -170, -140
n_bags = 150
dt = 21600
n_steps = ds.dims['time']
lats = ds["latitude"].values
lons = ds["longitude"].values

# === 3. Initialize particle positions randomly within the Ross Sea box ===
np.random.seed(42)
particles_lat = np.random.uniform(lat_min, lat_max, size=n_bags)
particles_lon = np.random.uniform(lon_min, lon_max, size=n_bags)

traj_lats = np.zeros((n_bags, n_steps))
traj_lons = np.zeros((n_bags, n_steps))
traj_lats[:, 0] = particles_lat
traj_lons[:, 0] = particles_lon

# === 4. Lagrangian simulation loop ===
for t in range(1, n_steps):
    uo = ds["uo"].isel(time=t-1, depth=0).values
    vo = ds["vo"].isel(time=t-1, depth=0).values
    interp_u = RegularGridInterpolator((lats, lons), uo, bounds_error=False, fill_value=np.nan)
    interp_v = RegularGridInterpolator((lats, lons), vo, bounds_error=False, fill_value=np.nan)
    coords = np.stack((particles_lat, particles_lon), axis=-1)
    u_vals = interp_u(coords)
    v_vals = interp_v(coords)
    mask = ~np.isnan(u_vals) & ~np.isnan(v_vals)

    delta_lat = np.zeros_like(particles_lat)
    delta_lon = np.zeros_like(particles_lon)
    delta_lat[mask] = (v_vals[mask] * dt) / 111320
    delta_lon[mask] = (u_vals[mask] * dt) / (40075000 * np.cos(np.radians(particles_lat[mask])) / 360)

    particles_lat[mask] += delta_lat[mask]
    particles_lon[mask] += delta_lon[mask]

    traj_lats[:, t] = particles_lat
    traj_lons[:, t] = particles_lon

# === 5. Setup KDE Grid ===
grid_lon = np.linspace(-180, -130, 200)
grid_lat = np.linspace(-80, -55, 200)
lon_grid, lat_grid = np.meshgrid(grid_lon, grid_lat)
grid_coords = np.vstack([lon_grid.ravel(), lat_grid.ravel()])

# === 6. Create animated KDE heatmap on polar map ===
fig = plt.figure(figsize=(10, 10))
ax = plt.axes(projection=ccrs.SouthPolarStereo())
ax.set_extent([-180, 180, -85, -55], crs=ccrs.PlateCarree())

ax.add_feature(cfeature.LAND, facecolor='cornsilk', zorder=0)
ax.add_feature(cfeature.OCEAN, facecolor='lightblue', zorder=0)
ax.coastlines(resolution='110m', linewidth=0.8)
ax.gridlines(draw_labels=False, linestyle=":", linewidth=0.5, color="gray")

kde_img = None
title = ax.set_title("", fontsize=16, fontfamily="arial", fontweight="bold")

def animate(t):
    global kde_img
    ax.clear()
    ax.set_extent([-180, 180, -85, -55], crs=ccrs.PlateCarree())
    ax.add_feature(cfeature.LAND, facecolor='cornsilk', zorder=0)
    ax.add_feature(cfeature.OCEAN, facecolor='lightblue', zorder=0)
    ax.coastlines(resolution='110m', linewidth=0.8)
    ax.gridlines(draw_labels=False, linestyle=":", linewidth=0.5, color="gray")

    # KDE calculation
    kde = gaussian_kde(np.vstack([traj_lons[:, t], traj_lats[:, t]]))
    kde_vals = kde(grid_coords).reshape(lon_grid.shape)

    kde_img = ax.pcolormesh(
        lon_grid,
        lat_grid,
        kde_vals,
        cmap="inferno",
        shading="auto",
        alpha=0.85,
        transform=ccrs.PlateCarree(),
        zorder=4
    )

    ax.set_title(f"🔥 Bag Density – Step {t}", fontsize=16, fontweight='bold', color='darkred')

ani = animation.FuncAnimation(fig, animate, frames=n_steps, interval=500, blit=False)
ani.save("Ross_Sea_Bags_Density.gif", writer=animation.PillowWriter(fps=2))
plt.close(fig)

print("✅ Animation saved as 'Ross_Sea_Bags_Density.gif'")

C:\Users\raul.galdeano.ext\AppData\Local\Temp\ipykernel_17072\1649127087.py:14: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  n_steps = ds.dims['time']
c:\Users\raul.galdeano.ext\AppData\Local\anaconda3\envs\MOHID_Lagrangian_environment\Lib\site-packages\cartopy\mpl\geoaxes.py:527: UserWarning: Glyph 128293 (\N{FIRE}) missing from font(s) DejaVu Sans.
  super()._update_title_position(renderer)
c:\Users\raul.galdeano.ext\AppData\Local\anaconda3\envs\MOHID_Lagrangian_environment\Lib\site-packages\cartopy\mpl\geoaxes.py:524: UserWarning: Glyph 128293 (\N{FIRE}) missing from font(s) DejaVu Sans.
  return super().draw(renderer=renderer, **kwargs)
c:\Users\raul.galdeano.ext\AppData\Local\anaconda3\envs\MOHID_Lagrangian_environment\Lib\site-packages\cartopy\mpl\geoaxes.py:524: UserWarning: Glyp

✅ Animation saved as 'Ross_Sea_Bags_Density.gif'
